In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import cv2
import os
import numpy as np
from keras.callbacks import ReduceLROnPlateau
from matplotlib import pyplot as plt
from keras.callbacks import ReduceLROnPlateau

In [ ]:
labels = ['BACTERIA','VIRUS', 'NORMAL']
image_size = 200

def get_training_data(data_dir):
    images_labels_data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (image_size, image_size))
                images_labels_data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(images_labels_data)

In [ ]:
train = get_training_data('x-rays/train')
test = get_training_data('x-rays/test')
val = get_training_data('x-rays/val')

In [ ]:
x_train = []
y_train = []

x_val = []
y_val = []

x_test = []
y_test = []

for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

for feature, label in test:
    x_test.append(feature)
    y_test.append(label)

for feature, label in val:
    x_val.append(feature)
    y_val.append(label)

In [ ]:
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255
x_test = np.array(x_test) / 255

In [ ]:
x_train = x_train.reshape(-1, image_size, image_size, 1)
y_train = np.array(y_train)

x_val = x_val.reshape(-1, image_size, image_size, 1)
y_val = np.array(y_val)

x_test = x_test.reshape(-1, image_size, image_size, 1)
y_test = np.array(y_test)

In [ ]:
datagen= ImageDataGenerator(

)

datagen.fit(x_train)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape = (image_size, image_size, 1)),
    tf.keras.layers.MaxPool2D((2, 2),),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2),),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu',),
    tf.keras.layers.MaxPool2D((2, 2),),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2),),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(3 , activation='softmax')
])


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
training_history = model.fit(datagen.flow(x_train, y_train, batch_size=10), epochs=7, validation_data=datagen.flow(x_val, y_val))

In [ ]:
plt.plot(training_history.history['accuracy'])
plt.plot(training_history.history['val_accuracy'])
plt.title('accuracy of model')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='lower left')

In [ ]:
plt.plot(training_history.history['loss'])
plt.plot(training_history.history['val_loss'])
plt.title('loss of model')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.show

In [ ]:
test_loss, test_acc = model.evaluate(datagen.flow(x_test, y_test))
print(test_acc)